<h1>Example notebook</h1>
<p>Welcome to this example notebook for training a model of a Convolutional Neural Network. This trained model can then be exporting it to an ONNX file, which is an exchange file format and stands for "Open Neural Network Exchange".</p>
<p>The training and export procedures are configured using a Configuration instance and two Dataset instances. Various methods with use these instances to initialize, train and export a Convolutional Neural Network. The classes and methods are defined in the "inference_training.py" file.</p>

<h2>Imports</h2>
<p>First we will import all functions that we will use in this notebook.</p>

In [1]:
from inference_training import Configuration, ImageDataset
from inference_training import initCudaEnvironment, createTransforms
from inference_training import drawImageAndFeatureMasks
from inference_training import exportOnnxModel, writeONNXMeta, loadONNX
from inference_training import trainModel, saveModel, loadModel
from inference_training import createModelInstance, testInference
from inference_training import logger

<h2>Training environment</h2>
<p>With this function we will initialize the cude environment, if present on our workstation.</p><p></p>In case multiple gpu's (cuda devices) are present on your workstations, you can adjust the parameters below to select how many and which ones should be used.</p>


In [2]:
initCudaEnvironment(numCudaDevices=1,
                    visibleCudaDevices="0",
                    clearCudaDeviceCount=False)

<h1>Configuration</h1>
<p>Here we initialize a configuration instance, used by the datasets and the creation and training of a model of a Convolution Neural Network.</p>

In [3]:
# train on the GPU or on the CPU, if a GPU is not available
config = Configuration()
print("Using Device: " + str(config.device))

<p>Here we must specify the path to both of your datasets; the test and train set.</p><p>Note that on the Windows operating system, the path is often copied with '\', which should be converted to '\\\\' or '/' in python code.</p>

In [ ]:
import os 

trainDirectory = "/path/to/train/dataset/"
testDirectory = "/path/to/test/dataset/"

assert os.path.isdir(trainDirectory), f'Train directory does not exists! {trainDirectory}';
assert os.path.isdir(testDirectory), f'Test directory does not exists! {testDirectory}';

config.setDatasetPaths(trainPath=trainDirectory, testPath=testDirectory)
config.setFilePrefix("")
config.setModelName("example_model")
config.setInputSizes(inputWidth=200, inputHeight=200)
config.setInputCellSize(cellSizeM=0.25, minCellSizeM=0.1, maxCellSizeM=0.5)
config.setAutoLimitLabel(True)

print("Model name: " + str(config.getModelName()))
print("Version: " + str(config.version))

<p>In this step we will have to define some important parameters; <li><ul>the number of channels of the input images. Normally this is 3, but images can be concatenated when more are exported.</ul><ul>the maximum number of classes that are present in the exported datasets; In case more classes are present in any of the datasets, an error will be thrown when validating the dataset.</ul><ul>Whether features are allowed to overlap or not.</ul<ul>The maximum amount of features per image. By default it is 250, close the maximum byte value. The used model will define the upper limit of this parameter due to limitations of the mask image.</ul><ul>The "reuseModel" parameter is used to indicate whether to open a previously trained and stored PyTorch model, if present. Otherwise a new model with default weights will be initialized.</ul></li></p>

In [5]:
numberOfClasses = 1

config.setModelInfo(channels=3, #
                    numClasses=numberOfClasses+1,  # ( +1 background)
                    bboxOverlap=True, #
                    bboxPerImage=250, #
                    reuseModel=False) #
config.setEpochs(10)


<p>In this configuration step, we define parameters that will be stored in the exported ONNX file. The ONNX file is an exchange file format that can be imported into the Tygron Platform. In later steps of this notebook, we will call methods that will export the trained PyTorch model to an ONNX model using the ONNX libraries.</p>
    
<p>In this step we configure certain meta data properties, such as legend entries, thresholds and tensornames, in such a s way that the Tygron Platform can identify and configure Tygron data objects directly.</p>

In [6]:
description = "Model description"
config.setOnnxInfo(producer="Tygron", description=description)

#Legend entries used by the label result type of an Inference Overlay in the Tygron Platform
config.clearLegendEntries()
config.addLegendEntry("Background", 0, "#00000000")
config.addLegendEntry("Foliage", 1, "#00ffbf")

missingLegendEntries = numberOfClasses + 1 - len(config.getLegendEntries())
if missingLegendEntries > 0:
    logger.warning(str(missingLegendEntries) + " Legend Entries are not yet defined! Add more legend entries using \"config.addLegendEntry(name, classIndex, webcolor)\"")

# Inference Overlay model parameters used when applying this ONNX model.
config.setOnnxMetaData(scoreThreshold=0.2,
                       maskThreshold=0.3,
                       strideFraction=0.5)

#Name of the input tensor and the batch amount set when exporting this model to an ONNX file.
config.setTensorInfo(tensorName='input_A:RGB_normalized', batchAmount=1)

<h1>Datasets</h1>
<p>In this step we initialize the image datasets. These take the configuration as input, as well as a boolean indicating it is a train set or test set.</p>
<p>The final argument are transforms, which are applied to the input images. This results in a larger dataset and a model that can cope better with minor changes in images. You can also define your own list of transformations, just be careful with using random transformations.</p>

In [7]:
trainingDataset = ImageDataset(config, True, createTransforms(True))
testDataset = ImageDataset(config, False, createTransforms(False))

logger.info("Train Image count: "+str(trainingDataset.__len__()))
logger.info("Test Image count: "+str(testDataset.__len__()))

trainingDataset.validate()
testDataset.validate()

logger.info("Pytorch model name " + config.getPytorchModelFileName())
logger.info("Onnx file name " + config.getOnnxFileName())


<p>Here we simply test an image of our training dataset to verify that we are using the correct input image. We also highlight each inidividal feature provided by the mask image.</p>

In [8]:
imageNumber = 5
print(trainingDataset.getLabels(imageNumber))
drawImageAndFeatureMasks(config, trainingDataset, imageNumber)

<h2>Training and saving the model</h2>
<p>In this step we will train and save the model. In the code below, the model and training parameters are automatically created using the configuration file. If you want to specify your own model and training procedure, please check the train model function in the 'inference_training.py' file.</p>
<p>Once trained, the model is saved using the configured pytorch model file name.</p>

In [9]:
model = trainModel(config, trainingDataset, testDataset)
saveModel(config, model, path=config.getPytorchModelFileName())

<h2>Validating the trained model</h2>
<p>In this step we can validate our trained network by testing it on an image of our test dataset.</p>

In [10]:
model.eval()
testPrediction = testInference(config, model=model,
                               dataset=testDataset, imageNumber=88)

<h2>Export to an ONNX file</h2>
<p>In this part we will export the onnx model, set the metadata (for use in the Tygron Platform)</p><p>Using the configuration instance, we can simply export the trained model to an ONNX file using this function:</p>

In [11]:
exportOnnxModel(config, model)

<p>Here we write the meta data using the configuration as input above. This meta data will be used by the Tygron Platform to automatically fill in legend entries, model parameters and input parameters.</p>

In [12]:
writeONNXMeta(config)

<h2>Validating the ONNX file</h2>
<p>In this last step, we will try to load the exported ONNX file and see if the meta data is present.</p>

In [13]:
onnx_model = loadONNX(config)
print(f"metadata_props={onnx_model.metadata_props}")